In [1]:
# https://stackoverflow.com/questions/69515086/beautifulsoup-attributeerror-collections-has-no-attribute-callable
import collections
collections.Callable = collections.abc.Callable
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
lst = ee.ImageCollection('MODIS/006/MOD11A1')

In [3]:
i_date = '2017-01-01'
f_date = '2020-01-01'
# get specific band, span of dates
# lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)
lst = lst.select('LST_Day_1km').filterDate(i_date, f_date)

In [4]:
# coords of Dhaka Bangladesh = 23.8103, 90.4125 
# lst = ee.ImageCollection('MODIS/006/MOD11A1')
bd_lon = 23.8013
bd_lat = 90.4125
bd_poi = ee.Geometry.Point(bd_lon, bd_lat)


In [5]:
scale = 1000 #in meters
lst_bd_poi = lst.getRegion(bd_poi, scale).getInfo()
lst_bd_poi[:5]

[['id', 'longitude', 'latitude', 'time', 'LST_Day_1km'],
 ['2017_01_01', 23.80086345274672, 90.41094177020925, 1483228800000, None],
 ['2017_01_02', 23.80086345274672, 90.41094177020925, 1483315200000, None],
 ['2017_01_03', 23.80086345274672, 90.41094177020925, 1483401600000, None],
 ['2017_01_04', 23.80086345274672, 90.41094177020925, 1483488000000, None]]

In [34]:
roi = bd_poi.buffer(1e6)

In [35]:
lst_img = lst.mean()


In [36]:
from IPython.display import Image
url = lst_img.getThumbUrl({
    'min': 0, 'max': 100, 'dimensions': 512, 'region': roi,
    'palette': ['blue', 'yellow', 'orange', 'red']
})
print(url)
Image(url=url)
# https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/acebb2b62ea87dbf2949743b09dea816-71e45ea5299813305ee0bd1f8a1f2b1b:getPixels

EEException: Image.visualize: Cannot provide a palette when visualizing more than one band.

In [17]:
# contains administrative boundaries
countries = ee.FeatureCollection('FAO/GAUL/2015/level10').select('ADM0_NAME')
bangladesh = countries.filter(ee.Filter.eq('ADM0_NAME', 'France'))
lst_bd = lst_img.clip(bangladesh)
url = lst_bd.getThumbUrl({
    'min': 0, 'max': 2500, 'region': roi, 'dimensions': 512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
})

AttributeError: 'ImageCollection' object has no attribute 'clip'